In [ ]:
#!pip install kaggle
#!pip install missigno
#!pip install pandas-profiling
import os
import networkx as nx
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
import seaborn as sns 
import matplotlib.pyplot as plt
import missingno as msno
import regex as re
from pandas_profiling import ProfileReport

Data from: https://www.kaggle.com/leomauro/smmnet

# Datenbeschaffung & Aufbereitung

## Setup kaggle api

In [ ]:
# Create .kaggle path in user dir if not exists
if not os.path.exists(str(Path.home())+"\.kaggle"):
    os.makedirs(str(Path.home())+"\.kaggle")

Go to www.<span>kaggle.<span>com/**your_username**/account and download a new api-token. 
After downloading the json file needs to be put into the C:\Users\\**username**\\.kaggle directory.

## Download dataset

In [ ]:
# Check if folder already exists
if not os.path.isdir('./data'):
    
    # if not create data folder
    os.makedirs('./data') 
    
    # download csv's
    !kaggle datasets download -d "leomauro/smmnet" -p "./data"

## Unzip files

In [ ]:
file_list = ["clears.csv","course-meta.csv","courses.csv","likes.csv","players.csv","plays.csv","records.csv"]
files_exist = []

# Check if all csv's exist
for file in file_list:
    files_exist.append(os.path.isfile("./data/" + file))
    
# if not create csv's
if not all(files_exist):
    with zipfile.ZipFile("./data/smmnet.zip", 'r') as zip_ref:
        zip_ref.extractall("./data") 

## Import files as Pandas DataFrame

In [ ]:
clears = pd.read_csv("./data/clears.csv", delimiter = "\t")
course_meta = pd.read_csv("./data/course-meta.csv", delimiter = "\t") # index_col=["id","firstClear"]
courses = pd.read_csv("./data/courses.csv", delimiter = "\t")
likes = pd.read_csv("./data/likes.csv", delimiter = "\t")
players = pd.read_csv("./data/players.csv", delimiter = "\t")
plays = pd.read_csv("./data/plays.csv", delimiter = "\t")
records = pd.read_csv("./data/records.csv", delimiter = "\t")

dfs = {"clears":clears, "course_meta":course_meta, "courses":courses, "likes":likes, "players":players,"plays":plays, "records":records}

In [ ]:
courses.head()

# Data Validation
In this Kaggle Dataset, we provide over 115 thousand games maps created on Super Mario Maker with over 880 thousand players which performed over 7 millions of interactions on these maps. By interactions, this means that a player can: (1) create a game map; (2) play a map created by other players; if a player completes the challenge of the game map, he/she (3) "cleared" the map; also can be the (4) first clear; beat the (5) time record of a map; (6) at any time, the player can "like" a game map. Note, this dataset present temporal changes over time for each game map by a period of three months.

The data was extracted from supermariomakerbookmark.nintendo.net, the game website. Now it is publicly to everyone play, explore and research. This dataset serves as a good base for learning models, including, but not limited to, Player Modeling (e.g., player experience), Data Mining (e.g., prediction, and find patterns), and Social Network Analysis (e.g., community detection, link prediction, ranking).

In [ ]:
%%capture
# Check if folder already exists
if not os.path.isdir('./df_reports'):
    
    # if not create data folder
    os.makedirs('./df_reports')  
    for name, df in dfs.items():
        ProfileReport(df, title=f"Pandas Profiling Report, df = {name}").to_file(f"./df_reports/report_{name}.html")

In [ ]:
def preprocessing(dfs, drop=False, fill=True):
    '''Preprocessing function for Super Mario Maker Dataset.
    - Changes datatypes where necessary
    - Runs multiple asserts to check if quantity/quality of data is correct
    - Deals with missing firstClear values
    - Corrects "clears"-Values where necessary
    - Deals with missing values

    Arguments
    ---------
    dfs: dict, Dictionary of DataFrames (Super Mario Maker DataFrames from Kaggle)
    drop: bool, Removes rows or courses if deemed defect, default False
    fill: bool, Fills firstClear Values if clear can be found in clears-df else sets clears to 0, default True

    Returns
    -------
    dfs : returns dictionary of dataframes
    '''

    # multiple asserts
    assert len(players) >= 880_000, "Number of players is less than expected"
    assert len(courses) >= 115_000, "Number of courses is less than expected"
    assert (course_meta.attempts >= course_meta.clears).all(), "At least one occurance of clears > attempts detected"
    assert len(players) == len(players.id.unique()), "Player id not unique"
    assert len(courses) == len(courses.id.unique()), "Course id not unique"

    # add correct dtype to timestamps, and assert values
    for name, df in dfs.items():
        if "catch" in df.columns:
            df.catch = pd.to_datetime(df.catch)
            assert df.catch.between("2017-11-15","2018-04-11").all(), f"Detected catch timestamp out of range '2017-11-16' - '2018-04-10' in df {name}"
        elif "creation" in df.columns:
            df.creation = pd.to_datetime(df.creation)
            assert (df.creation <= "2018-04-10").all(), f"Detected creation date more recent than 2018-04-10 in df {name}"

        # test for negative numbers in numeric variables
        numerics = ['int64', 'float64']
        num_cols = df.select_dtypes(include=numerics).columns
        for col in num_cols:
            assert (df[col].values >= 0).all(), f"At least one occurance of negative values detected in df {name}"

        # test if id is in allowed format
        r = re.compile("[A-Z|0-9]{4}-[A-Z|0-9]{4}-[A-Z|0-9]{4}-[A-Z|0-9]{4}")
        if "id" in df.columns and name != "players":
            assert df.id.str.fullmatch("[A-Z|0-9]{4}-[A-Z|0-9]{4}-[A-Z|0-9]{4}-[A-Z|0-9]{4}").all(), f"At least one id in wrong format detected in df {name}"

    # sort course_meta and only keep most recent
    course_meta_sort = course_meta.sort_values(by="catch",ascending=False)
    course_meta_uniq = course_meta_sort.groupby(by="id").first().reset_index()
    dfs['course_meta'] = course_meta_uniq
    
    if fill:
        # drop courses with clears > 0 and fistclear == NaN
        invalid_courses = course_meta[(course_meta.clears != 0) & (course_meta.firstClear.isna())]
        # sort by catch, ascending, get first player for every id and import it into courses_meta
        candidate_firstClear = clears[clears.id.isin(list(invalid_courses.id.values))]
        candidate_clean = candidate_firstClear.sort_values(by="catch", ascending=True).groupby(by='id').first().reset_index()

        def func(row):
            if row.id in (candidate_clean.id.values): 
                return candidate_clean[candidate_clean.id == row.id].player

        course_meta['firstClear'] = course_meta.apply(lambda x: func(x), axis=1)

        # set clears to 0 for courses that still have no first clear
        course_meta.loc[(course_meta.clears != 0) & (course_meta.firstClear.isna()), "clears"] = 0
        
        # check if clears == 0 than firstClear == NaN
        assert ((course_meta.clears == 0) & (course_meta.firstClear.isna())).any(), 'Detected courses with clears > 0 and firstClear = NaN'
        
        # check if sum clears == 0 = firstClear == NaN
        assert (len(course_meta.clears == 0) == len(course_meta.firstClear.isna())), 'Missing values in firstClear collumn do not correspond with clears collumn'

    # course_meta[course_meta.tag == "Thumbnail"] # should thumbnail be possible as a tag?
    
    # TODO: Change from simple drop to "database drop", remove id's and players from all datasets
    if drop: 
        # drop courses with firstClear NaN and clears > 0 (df course_meta)
        course_meta.drop(course_meta[(course_meta.clears != 0) & (course_meta.firstClear.isna())].index, inplace=True)
        # drop courses with maker NaN (df courses)
        courses.drop(courses[courses.maker.isna()].index, inplace=True)
        # drop players with no name (df players)
        players.drop(players[players.name.isna()].index, inplace=True)
        # drop courses where title is NaN
        courses.drop(courses[coureses.title.isna()].index, inplace=True)
            
    return dfs
dfs = preprocessing(dfs, drop=True, fill=True)    

In [ ]:
for name, df in dfs.items():
    if df.isnull().sum().sum() > 0:
        print(f"{name}:")
        print(df.isnull().sum())
        print("\n")

# EDA

In [ ]:
for name, df in dfs.items():
    msno.matrix(df, figsize=(8,2))
    plt.show()

In [ ]:
for name, df in dfs.items():
    if "catch" in df.columns:
        col = "catch"
    elif "creation" in df.columns:
        col = "creation"
    else:
        continue
    plt.figure(figsize=(8,2))
    sns.histplot(df[col]).set_title(f"Distribution of Timestamps for '{col}' in df {name}")
    plt.show()

# Social Network Analysis